<a href="https://colab.research.google.com/github/axel-sirota/pre-trained-nlp-models/blob/main/module3/PreTrainedNLP_Mod3Demo2_Fairness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler




In [26]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O income.csv https://www.dropbox.com/scl/fi/09urkrujttqv7wpfw5r4r/adult.csv?rlkey=s9bjnhf7q88w5y2alij7sxgdm&dl=0
fi


Overwriting get_data.sh


In [27]:
!bash get_data.sh

--2024-01-09 04:49:34--  https://www.dropbox.com/scl/fi/09urkrujttqv7wpfw5r4r/adult.csv?rlkey=s9bjnhf7q88w5y2alij7sxgdm
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uce350cd764b1909575cc685618c.dl.dropboxusercontent.com/cd/0/inline/CK_ODUb3ye3TIzf2g5Yii2MNYOuk5mRX6W7CXwBb92nDcvpK5zsxwBoMn1tUEVI6i0rWv0lw-xmm8rt3D8m0cPwK9xQjgqevh3kmPCvj719kqkAoFdfgr3fkw7kKc0SZpcym1Fi4NWY7z7dIc0Qnwy5o/file# [following]
--2024-01-09 04:49:36--  https://uce350cd764b1909575cc685618c.dl.dropboxusercontent.com/cd/0/inline/CK_ODUb3ye3TIzf2g5Yii2MNYOuk5mRX6W7CXwBb92nDcvpK5zsxwBoMn1tUEVI6i0rWv0lw-xmm8rt3D8m0cPwK9xQjgqevh3kmPCvj719kqkAoFdfgr3fkw7kKc0SZpcym1Fi4NWY7z7dIc0Qnwy5o/file
Resolving uce350cd764b1909575cc685618c.dl.dropboxusercontent.com (uce350cd764b1909575cc685618c.dl.dropboxusercontent.com)... 162.125.3.15, 2620:1

In [28]:
# Load the dataset
dataset = pd.read_csv('income.csv')
# Encode categorical columns
categorical_columns = ['workclass', 'education', 'occupation', 'sex', "marital.status", "relationship", "race", "income", "native.country"] # Add other categorical columns
for col in categorical_columns:
    dataset[col] = LabelEncoder().fit_transform(dataset[col])

# Normalize numerical columns
numerical_columns = ['age', 'fnlwgt', 'education.num', "hours.per.week"]
scaler = StandardScaler()
dataset[numerical_columns] = scaler.fit_transform(dataset[numerical_columns])

# Split into features and labels
X = dataset.drop('income', axis=1)  # Assuming 'income' is the label column
y = dataset['income']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_tf = tf.convert_to_tensor(np.asarray(X_train.values).astype(np.float32))
X_test_tf = tf.convert_to_tensor(np.asarray(X_test.values).astype(np.float32))
y_train_tf = tf.convert_to_tensor(np.asarray(y_train.values).astype(np.float32))
y_test_tf = tf.convert_to_tensor(np.asarray(y_test.values).astype(np.float32))

In [29]:
X_test

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
14160,-0.702455,4,0.860645,15,-0.031360,2,6,0,4,1,0,0,-0.035429,39
27048,-1.435581,4,2.368589,15,-0.031360,4,12,3,4,1,0,0,-2.060174,39
28868,-0.775768,4,0.141695,15,-0.031360,2,13,5,4,0,0,0,-0.035429,39
5667,-0.262580,4,-0.774881,1,-1.197459,4,12,1,4,1,0,0,0.369519,39
7827,-1.362269,4,-1.284969,15,-0.031360,4,1,3,4,0,0,0,-0.845327,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,-0.849080,6,-0.363590,11,-0.420060,2,5,0,4,1,0,1573,2.394264,39
24534,1.790173,4,-0.020307,11,-0.420060,4,4,1,4,1,0,0,-0.035429,39
18080,1.276984,4,-0.755818,12,1.523438,2,10,0,4,1,0,0,-0.035429,39
10354,-1.215643,0,-0.066390,8,0.357340,4,0,3,1,1,0,0,-1.655225,39


In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(X_train_tf, y_train_tf, epochs=10, batch_size=32)


Epoch 1/10
814/814 [==============================] - 2s 2ms/step - loss: 2.7282 - accuracy: 0.7953
Epoch 2/10
814/814 [==============================] - 2s 2ms/step - loss: 0.9475 - accuracy: 0.8166
Epoch 3/10
814/814 [==============================] - 2s 2ms/step - loss: 0.5981 - accuracy: 0.8248
Epoch 4/10
814/814 [==============================] - 2s 2ms/step - loss: 0.4014 - accuracy: 0.8301
Epoch 5/10
814/814 [==============================] - 2s 2ms/step - loss: 0.4885 - accuracy: 0.8261
Epoch 6/10
814/814 [==============================] - 2s 3ms/step - loss: 0.3997 - accuracy: 0.8269
Epoch 7/10
814/814 [==============================] - 2s 3ms/step - loss: 0.3474 - accuracy: 0.8318
Epoch 8/10
814/814 [==============================] - 2s 2ms/step - loss: 0.3519 - accuracy: 0.8300
Epoch 9/10
814/814 [==============================] - 2s 2ms/step - loss: 0.3480 - accuracy: 0.8280
Epoch 10/10
814/814 [==============================] - 2s 2ms/step - loss: 0.6984 - accuracy: 0.8223

In [32]:
# Filter the test set by gender
male_test = X_test[X_test['sex'] == 0]  # Assuming male is encoded as 1
y_male_test = y_test.loc[male_test.index]
female_test = X_test[X_test['sex'] == 1]  # Assuming female is encoded as 0
y_female_test = y_test.loc[female_test.index]

# convert to tensors

male_test_tf = tf.convert_to_tensor(np.asarray(male_test.values).astype(np.float32))
female_test_tf = tf.convert_to_tensor(np.asarray(female_test.values).astype(np.float32))
y_male_test_tf = tf.convert_to_tensor(np.asarray(y_male_test.values).astype(np.float32))
y_female_test_tf = tf.convert_to_tensor(np.asarray(y_female_test.values).astype(np.float32))

# Evaluate performance
male_performance = model.evaluate(male_test_tf, y_male_test_tf)
female_performance = model.evaluate(female_test_tf, y_female_test_tf)

# Compare metrics to detect bias
print(f'Male income >=50K % : {male_performance[1] * 100}')
print(f'Female income >=50K % : {female_performance[1] * 100}')

68/68 [==============================] - 0s 2ms/step - loss: 0.2200 - accuracy: 0.9108
Male income >=50K % : 79.79357838630676
Female income >=50K % : 91.08220934867859


In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Assuming model is your trained TensorFlow model
predictions = model.predict(X_test)
# Convert predictions to binary labels (0 or 1)
predicted_labels = (predictions > 0.5).astype(int)

def calculate_fairness_metrics(y_true, y_pred, sensitive_feature):
    groups = sensitive_feature.unique()
    metrics = {}

    for group in groups:
        group_mask = (sensitive_feature == group)
        group_y_true = y_true[group_mask]
        group_y_pred = y_pred[group_mask]

        tn, fp, fn, tp = confusion_matrix(group_y_true, group_y_pred).ravel()

        # Equal Opportunity: True Positive Rate
        tpr = tp / (tp + fn)
        metrics[f'TPR_{group}'] = tpr

        # Predictive Equality: False Positive Rate
        fpr = fp / (fp + tn)
        metrics[f'FPR_{group}'] = fpr

        # Accuracy
        accuracy = accuracy_score(group_y_true, group_y_pred)
        metrics[f'Accuracy_{group}'] = accuracy

    return metrics

# Example Usage
calculate_fairness_metrics(y_test, predicted_labels, X_test['sex'])


204/204 [==============================] - 1s 4ms/step


{'TPR_1': 0.44973138910207217,
 'FPR_1': 0.053647366699378474,
 'Accuracy_1': 0.7979357798165138,
 'TPR_0': 0.2863247863247863,
 'FPR_0': 0.013027618551328817,
 'Accuracy_0': 0.910822108685555}